# Buy it Again

- https://www.kaggle.com/code/lichtlab/do-customers-buy-the-same-products-again
    



In [1]:
import os
import gc
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy.optimize import curve_fit
from joblib import dump, load
import scipy.stats as stats
import seaborn as sns
tqdm.pandas()

import sys
sys.path.insert(0, '../../')

from utils.preprocess import *
from buyitagain.preprocess_bia import *
from buyitagain.utils_bia import *

In [2]:

folder = '../../../data/'

art_df = pd.read_csv(os.path.join(folder, "articles.csv"))
cus_df = pd.read_csv(os.path.join(folder, "customers.csv"))
trans_df = pd.read_csv(os.path.join(folder, "transactions_train.csv"))

trans_df = make_weeknum_col(trans_df)
# week_num 0 is latest week of the transaction
train_df, val_df = split_train_valid(trans_df, 0)
# val_df = valid2submission(val_df)


In [3]:
col = 'idxgrp_idx_prdtyp'
rcp_threshold = 0.2
min_num_purchased = 4

In [4]:
use_cols = ['t_dat', 'customer_id', 'article_id']
train_df = train_df[use_cols]

# https://www.kaggle.com/code/lichtlab/do-customers-buy-the-same-products-again
art_df['idxgrp_idx_prdtyp'] = art_df['index_group_name'] + '_' + art_df['index_name'] + '_' + art_df['product_type_name'] 

use_cols = ['article_id', 'prod_name', 'idxgrp_idx_prdtyp']
sample_art_df = art_df[use_cols]
train_df = pd.merge(train_df, sample_art_df, how='left', on='article_id')

train_df['t_dat'] = pd.to_datetime(train_df['t_dat'])
train_df['trans_idx'] = train_df.index


In [5]:

col_train_df = train_df.drop_duplicates(subset=["t_dat", "customer_id", col], keep='last')


In [6]:

col_train_df = make_time_interval_col(col_train_df, col)

col_g_df = groupby_cid_artinfo(col_train_df, col)

col_g_df = make_rcp_df(col_g_df, col)

rp_col = get_repeat_purchasable(col_g_df, rcp_threshold, denom_customer_num=30)

use_cols = ['article_id', 'prod_name', 'idxgrp_idx_prdtyp']
if col == 'article_id':
    rp_col = pd.merge(rp_col, art_df[use_cols], 
                      on=col, how='left')
    rp_col.sort_values('rcp', ascending=False).head(23)
    
col_train_df = col_train_df[col_train_df[col].isin(rp_col[col])]

rp_col[['shape', 'rate']]= rp_col.apply(lambda x: get_shape_rate(x, col_train_df, col), axis=1, result_type='expand')

rp_cid_col_df = get_rp_cid_col_df(col_train_df, col)

rp_cid_col_df = rp_cid_col_df[rp_cid_col_df.cnt >= min_num_purchased]
rp_cid_col_df = pd.merge(rp_cid_col_df, rp_col, on=col, how='left')


rp_cid_col_df = make_date_info_col(val_df, rp_cid_col_df)



재구매가능 상품의 최소 고객수: 30


/var/folders/fl/l7bfgt6j3lz29pwdkr0_537r0000gn/T/ipykernel_76202/1481132115.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rp_col[['shape', 'rate']]= rp_col.apply(lambda x: get_shape_rate(x, col_train_df, col), axis=1, result_type='expand')
/var/folders/fl/l7bfgt6j3lz29pwdkr0_537r0000gn/T/ipykernel_76202/1481132115.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rp_col[['shape', 'rate']]= rp_col.apply(lambda x: get_shape_rate(x, col_train_df, col), axis=1, result_type='expand')
/Users/nys/Desktop

In [7]:
rp_cid_col_df[['lambda_min','lambda_max']] = rp_cid_col_df.apply(lambda x: get_lambda(x), axis=1, result_type='expand')

rp_cid_col_df['R_ac_min'] = rp_cid_col_df['lambda_min'].apply(lambda x: calc_hpp(x,100))
rp_cid_col_df['R_ac_max'] = rp_cid_col_df['lambda_max'].apply(lambda x: calc_hpp(x,100))

# purchase product density
rp_cid_col_df['ppd_min'] = rp_cid_col_df['rcp'] * rp_cid_col_df['R_ac_min']
rp_cid_col_df['ppd_max'] = rp_cid_col_df['rcp'] * rp_cid_col_df['R_ac_max']

rp_cid_col_df = rp_cid_col_df.sort_values('ppd_max', ascending=False)



In [8]:
cur_cus_id = rp_cid_col_df.iloc[0]['customer_id']
col_train_df[col_train_df.customer_id == cur_cus_id].sort_values('t_dat').tail(10)


,customer_id,idxgrp_idx_prdtyp,t_dat,article_id,prod_name,trans_idx,idxgrp_idx_prdtyp_lastdate,idxgrp_idx_prdtyp_dist
20994392,fe3026d45f6a22e728aee2b61cd94ac40772003813cef2...,Ladieswear_Ladieswear_Dress,2020-08-03,817181005,Sydney linen shirt dress,29878630,2020-08-01,2.0
20994393,fe3026d45f6a22e728aee2b61cd94ac40772003813cef2...,Ladieswear_Ladieswear_Dress,2020-08-05,890453001,Lorraine dress,29986793,2020-08-03,2.0
20994394,fe3026d45f6a22e728aee2b61cd94ac40772003813cef2...,Ladieswear_Ladieswear_Dress,2020-08-12,803993007,Dorsia dress(1),30293696,2020-08-05,7.0
20994395,fe3026d45f6a22e728aee2b61cd94ac40772003813cef2...,Ladieswear_Ladieswear_Dress,2020-08-13,852369006,Dream,30333251,2020-08-12,1.0
20994396,fe3026d45f6a22e728aee2b61cd94ac40772003813cef2...,Ladieswear_Ladieswear_Dress,2020-08-14,783775004,New York tunic dress,30368583,2020-08-13,1.0
20994397,fe3026d45f6a22e728aee2b61cd94ac40772003813cef2...,Ladieswear_Ladieswear_Dress,2020-08-18,846581001,Asta Dress (1),30488058,2020-08-14,4.0
20994398,fe3026d45f6a22e728aee2b61cd94ac40772003813cef2...,Ladieswear_Ladieswear_Dress,2020-08-23,865520002,Luca,30665162,2020-08-18,5.0
20994405,fe3026d45f6a22e728aee2b61cd94ac40772003813cef2...,Ladieswear_Ladieswear_Top,2020-08-23,767473006,ED Ruccola,30665163,2020-07-20,34.0
20994399,fe3026d45f6a22e728aee2b61cd94ac40772003813cef2...,Ladieswear_Ladieswear_Dress,2020-08-24,817491004,Bowie,30702880,2020-08-23,1.0
20994400,fe3026d45f6a22e728aee2b61cd94ac40772003813cef2...,Ladieswear_Ladieswear_Dress,2020-08-26,852365001,Carnival,30790660,2020-08-24,2.0


In [9]:
rp_cid_col_df.head()

,customer_id,idxgrp_idx_prdtyp,t_mean,t_purch,t_last,t_first,cnt,denom_customer,num_customer,rcp,...,t_min,t_max,t_pg_min,t_pg_max,lambda_min,lambda_max,R_ac_min,R_ac_max,ppd_min,ppd_max
507919,fe3026d45f6a22e728aee2b61cd94ac40772003813cef2...,Ladieswear_Ladieswear_Dress,2.100000,63.0,2020-08-26,2020-06-24,30,476374,236441,0.496335,...,21 days,27 days,84 days,90 days,0.366011,0.341613,0.637309,0.573040,0.316319,0.284420
149493,4ad7ee2ce850ba935c29c1390542bb39189f04392e0ac8...,Ladieswear_Ladieswear_Trousers,2.333333,14.0,2020-09-12,2020-08-29,6,520563,259551,0.498597,...,4 days,10 days,18 days,24 days,0.388088,0.280317,0.698988,0.428234,0.348513,0.213516
372085,b9b2f312ec49f847425925b47729ef5251a6a58d320d74...,Ladieswear_Ladieswear_Shirt,1.900000,19.0,2020-09-14,2020-08-26,10,185418,49942,0.269348,...,2 days,8 days,21 days,27 days,0.560668,0.398746,1.317111,0.730011,0.354761,0.196627
40903,1426b606af826fa9568b6fe75629d10bb6ff7ea40f9390...,Divided_Divided_Dress,3.050000,61.0,2020-09-14,2020-07-15,20,333925,134353,0.402345,...,2 days,8 days,63 days,69 days,0.327295,0.299312,0.537143,0.470683,0.216117,0.189377
365327,b637a3e7d8b0caa947aaefd609b8d84a9ee962cf0a52a5...,Ladieswear_Ladieswear_Dress,4.246914,344.0,2020-09-13,2019-10-05,81,476374,236441,0.496335,...,3 days,9 days,347 days,353 days,0.235924,0.231576,0.336876,0.328490,0.167203,0.163041


In [10]:
print("기존 길이:", len(val_df))
val_df = pd.merge(val_df, sample_art_df, how='left', on='article_id')


기존 길이: 240311


In [11]:
val_df[f'cid_{col}'] = val_df['customer_id'] + val_df[col]

threshold_df = rp_cid_col_df[(rp_cid_col_df.ppd_min > 0.001) |
                (rp_cid_col_df.ppd_max > 0.001)]

threshold_df[f'cid_{col}'] = threshold_df['customer_id'] + threshold_df[col]

correct_transaction = len(val_df[val_df[f'cid_{col}'].isin(threshold_df[f'cid_{col}'])])
print(correct_transaction/ len(val_df))




0.14087994307376692


/var/folders/fl/l7bfgt6j3lz29pwdkr0_537r0000gn/T/ipykernel_70519/2266363572.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  threshold_df[f'cid_{col}'] = threshold_df['customer_id'] + threshold_df[col]


In [12]:
reduced_val_df= val_df.drop_duplicates(subset=["customer_id", col], keep='last')
print("중복제거 길이:", len(reduced_val_df))


중복제거 길이: 163892


In [13]:

correct_transaction = len(reduced_val_df[reduced_val_df[f'cid_{col}'].isin(threshold_df[f'cid_{col}'])])
print(correct_transaction)
print(correct_transaction / len(reduced_val_df))



19236
0.11736997534962049
